# Exploratory Data Analysis & Preprocessing

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)

# suppress debugging warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
FOLDS = 3

## Data Preprocessing

### Load the dataset

In [ ]:
from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

from src import preprocessing

if "df_backup" not in globals():
    # takes 2-4min
    df = preprocessing.load_data()
    df_backup = df.copy()
else:
    df = df_backup.copy()  # type: ignore

### Brief Inspection

In [ ]:
df.info()

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df.dtypes)

In [ ]:
# View first few rows
print("First few rows:")
display(df.head())

# Check for missing values
print("\nMissing values:")
display(df.isnull().sum().where(lambda x: x > 0).dropna())

# Summary statistics
print("\nSummary statistics:")
display(df.describe(include='all'))

### Data Cleaning

In [ ]:
# Remove features that are independent of the target variable
INDEP_COLS = [
    'emp_title', 
    'id', 
    'member_id', 
    'policy_code', 
    'title', 
    'url',
]

# Remove columns that could cause data leakage
LEAKAGE_COLS = [
    'collection_recovery_fee', 
    'debt_settlement_flag_date',
    'debt_settlement_flag', 
    'deferral_term',
    'disbursement_method', 
    'funded_amnt_inv',
    'funded_amnt', 
    'hardship_amount',
    'hardship_dpd',
    'hardship_end_date',
    'hardship_flag',
    'hardship_flag',
    'hardship_length',
    'hardship_length',
    'hardship_loan_status',
    'hardship_reason',
    'hardship_start_date',
    'hardship_status',
    'hardship_type',
    'initial_list_status',
    'issue_d',
    'last_credit_pull_d', 
    'last_fico_range_high',
    'last_fico_range_low', 
    'last_pymnt_amnt',
    'last_pymnt_d', 
    'next_pymnt_d',
    'out_prncp_inv', 
    'out_prncp',
    'payment_plan_start_date',
    'pymnt_plan', 
    'recoveries', 
    'settlement_amount',
    'settlement_date',
    'settlement_percentage',
    'settlement_status',
    'settlement_term',
    'total_pymnt_inv', 
    'total_pymnt', 
    'total_rec_int',
    'total_rec_late_fee', 
    'total_rec_prncp', 
]

cols_to_drop = INDEP_COLS + LEAKAGE_COLS
df = preprocessing.drop_cols(df, cols=cols_to_drop)

In [ ]:
# Remove features with too many missing values
df = preprocessing.drop_sparse_cols(df, missing_rate=0.5)

In [ ]:
# Convert date columns to unix timestamps
date_cols = ['earliest_cr_line']
df = preprocessing.convert_dates(df, cols=date_cols)

In [ ]:
# handle emp_length
from src.preprocessing import extract_digits

df[df['emp_length'] == '< 1 year'] = 0
df = extract_digits(df, cols=['emp_length'])

In [ ]:
# Extract digits from string values
df = preprocessing.extract_digits(df, cols=['term', 'zip_code'])

In [ ]:
# Label Encoding for ordinal categorical variables
from src.feature_engineering import index_categories

grades = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
df = index_categories(df, 'grade', categories=grades)

sub_grades = [f'{g}{s}' for g in grades for s in range(1, 6)]
df = index_categories(df, 'sub_grade', categories=sub_grades)

In [ ]:
# Impute missing values
df = preprocessing.impute_missing_values(df)

In [ ]:
df.head()

### Categorical Encoding

In [ ]:
from src.feature_engineering import index_categories, frequency_encoding, onehot_encoding

# Encode target variable
df = index_categories(df, 'loan_status', categories=['Fully Paid', 'Charged Off'], drop=True)

In [ ]:
# Encode low-cardinality categorical variables using one-hot encoding
ONEHOT_CATS = [
    'application_type',
    'home_ownership',
    'purpose',
    'verification_status'
]

df = onehot_encoding(df, cols=ONEHOT_CATS)

In [ ]:
# Encode high-cardinality categorical variables using frequency encoding
FREQ_CATS = [
    'addr_state'
]

df = frequency_encoding(df, cols=FREQ_CATS)

### Drop highly correlated feature pairs

In [ ]:
from src import algebra, visualize
from src.feature_engineering import drop_corr_pairs

cols = df.columns.values
corr = algebra.correlation_matrix(df.drop(columns=['loan_status']))

# plot correlation matrix
visualize.correlation_matrix(corr)

df = drop_corr_pairs(df, corr)

removed = np.setdiff1d(cols, df.columns.values)
print(f"Removed columns:\n\t{"\n\t".join(sorted(removed))}")

### Scale feature values

In [ ]:
sorted(df.columns.values)

In [ ]:
from src.preprocessing import scale_features

df = scale_features(df, target='loan_status')

### Take another peak

In [ ]:
df.head()

### Handle data imbalance

In [ ]:
visualize.class_dist(df)

In [ ]:
from src import training

X_resampled, y_resampled = training.resample(df)

### Split the data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

### Feature Selection using RFE

In [ ]:
from src.model import create_model

# Takes ~20min on 16" M2 MBP
if "rfe" not in globals():
    from sklearn.feature_selection import RFECV

    rfe_model = create_model(eval_metric='logloss')

    # Create a model to determine the number of features for feature selection
    rfe = RFECV(
        estimator=rfe_model, cv=FOLDS, scoring='roc_auc', n_jobs=(-1))
    rfe.fit(X_train, y_train)
    n_features_to_select = rfe.n_features_

# Plot the number of features vs. cross-validation scores
visualize.plot_rfe(rfe)

rfe_score = rfe.score(X_test, y_test)
print(f"Optimal number of features to select: {rfe.n_features_}") # type: ignore
print(f"RFE Test Score: {rfe_score}")

In [ ]:
cols = X_train.columns[rfe.support_]
removed = X_train.columns[~rfe.support_]
X_train = pd.DataFrame(rfe.transform(X_train), columns=cols) # type: ignore
X_test = pd.DataFrame(rfe.transform(X_test), columns=cols) # type: ignore

print(f"Removed columns:\n\t{"\n\t".join(sorted(removed))}")

In [ ]:
X_test.head()

### Feature Correlations

In [ ]:
# calculate correlations between the features in X_train and the target variable y_train
df_train = pd.concat([X_train, y_train], axis=1)
correlations = (df_train.corr()['loan_status']
    .sort_values(ascending=False)
    .to_frame()
    .drop('loan_status')
)

visualize.correlations(correlations)

## Write the preprocessed data

In [ ]:
import joblib

joblib.dump((X_train, X_test, y_train, y_test), '../data/processed/accepted_2007_to_2018Q4.pkl')